In [28]:
from clickhouse_driver import Client
import json

In [34]:
!docker run -d --name clickhouse-server --ulimit nofile=262144:262144 -p 9000:9000 yandex/clickhouse-server

41908675385a95298f362114f5750f99138388ac94029d136e9f18e3b33b5e3b


Unable to find image 'yandex/clickhouse-server:latest' locally
latest: Pulling from yandex/clickhouse-server
ea362f368469: Already exists
38ba82a23e2b: Already exists
9b17d04b6c62: Already exists
5658714e4e8b: Already exists
6bde977a0bf8: Already exists
39053b27290b: Already exists
762d3d237065: Already exists
762d3d237065: Already exists
Digest: sha256:1cbf75aabe1e2cc9f62d1d9929c318a59ae552e2700e201db985b92a9bcabc6e
Status: Downloaded newer image for yandex/clickhouse-server:latest


In [35]:
# Подключение к серверу ClickHouse
client = Client(host='localhost',
                user='default',
                password='',
                port=9000)

try:
    result = client.execute('SHOW TABLES')
    print(result)
except Exception as e:
    print(f"Encountered an error: {e}")

[]


In [36]:
client.execute('SHOW TABLES')

[]

In [37]:
# Создание базы данных (если она не существует)
client.execute('CREATE DATABASE IF NOT EXISTS all_books')

[]

In [38]:
# Создание таблицы
# Создание основной таблицы 'books'
client.execute('''
CREATE TABLE IF NOT EXISTS all_book.books (
    `id` String,
    `UPC` String,
    `name` String,
    `description` String,
    `category` String,
    `rating` String,
    `price` Float32,
    `quantity` Int16
) ENGINE = MergeTree()
ORDER BY id
''')

print("Таблица создана успешно.")

ServerException: Code: 81.
DB::Exception: Database all_book doesn't exist. Stack trace:

0. DB::Exception::Exception(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&, int, bool) @ 0xa82d07a in /usr/bin/clickhouse
1. DB::DatabaseCatalog::assertDatabaseExistsUnlocked(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&) const @ 0x13519e5a in /usr/bin/clickhouse
2. DB::DatabaseCatalog::getDatabase(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&) const @ 0x1351c8f8 in /usr/bin/clickhouse
3. DB::InterpreterCreateQuery::createTable(DB::ASTCreateQuery&) @ 0x139ce684 in /usr/bin/clickhouse
4. DB::InterpreterCreateQuery::execute() @ 0x139d64ab in /usr/bin/clickhouse
5. ? @ 0x13d1434c in /usr/bin/clickhouse
6. DB::executeQuery(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&, std::__1::shared_ptr<DB::Context>, bool, DB::Query

In [ ]:
with open('./all_book.json', 'r', encoding='UTF-8-sig') as file:
    data = json.load(file)

In [ ]:
for doc in data:
    _id = doc['UPC']
    client.execute("""
    INSERT INTO all_book.books (
        UPC, name, description,
        category, rating, price, quantity
    ) VALUES""",
    [(_id,
     doc['name'] or "",
     doc['description'] or "",
     doc['category'] or "",
     doc['rating'] or "",
     doc['price'] or 0,
     doc['quantity'] or 0)])
print("Данные введены успешно.")

In [ ]:
# Проверка успешности вставки
result = client.execute("SELECT * FROM all_book.book")
print("Вставленная запись:", result[0])